# Load candidates

In [1]:
%load_ext autoreload
%autoreload 2


In [3]:
CACHE = '/home/anhphantq/vdb/cache_train'
import os
import pickle
from tqdm.notebook import tqdm
import numpy as np
import pandas as pd
import psutil

train_data = []

with open(CACHE + '/metadata', 'rb') as f:
  metadata = pickle.load(f)

num_frags = min(metadata['num_frags'], 1500)
max_session = metadata['max_session']

with tqdm(total= int (psutil.virtual_memory().total  / 1024  / 1024 / 1024), desc='RAM (GB)', position=0, leave= False) as rambar:

  for i in tqdm(range(0, num_frags)):

    train_data.append(pd.read_parquet(CACHE + f'/data_{i}.parquet'))
    if i == 0:
      print(train_data[-1].shape)
    rambar.n= int(psutil.virtual_memory().used / 1024  / 1024 / 1024)
    rambar.refresh()




RAM (GB):   0%|          | 0/125 [00:00<?, ?it/s]

  0%|          | 0/1029 [00:00<?, ?it/s]

(54032, 287)


In [4]:
train_data = pd.concat(train_data)

In [5]:
candidates = train_data
candidates.shape

(55994691, 287)

In [6]:
import pandas as pd
pd.set_option('display.max_columns', candidates.shape[1])

In [7]:
candidates

,user,item,num_clicks,num_carts,num_orders,item_degree,item_pr,recent_num_clicks,recent_num_carts,recent_num_orders,item_recent_degree,item_recent_pr,recent_day7_type0,recent_day7_type1,recent_day7_type2,recent_day6_type0,recent_day6_type1,recent_day6_type2,recent_day5_type0,recent_day5_type1,recent_day5_type2,recent_day4_type0,recent_day4_type1,recent_day4_type2,recent_day3_type0,recent_day3_type1,recent_day3_type2,recent_day2_type0,recent_day2_type1,recent_day2_type2,recent_day1_type0,recent_day1_type1,recent_day1_type2,item_glob_last_action,item_glob_first_action,item_glob_time_decay_sum,item_glob_time_decay_sum_click,item_glob_time_decay_sum_cart,item_glob_time_decay_sum_order,item_glob_item_glob_durability,num_cosub,coclick_sub_coef,cocart_sub_coef,coorder_sub_coef,coclick_time_decay,cocart_time_decay,coorder_time_decay,num_appearance,num_in_k_most_recent_items,num_happend_later,num_happend_before,happend_later_ratio,last_interact,cointeractions_sub_coef_sum,cointeractions_time_decay_sum,lincom_sub_coef_1_3_10,lincom_time_decay_1_3_10,lincom_sub_coef_1_10_3,lincom_time_decay_1_10_3,lincom_sub_coef_3_1_10,lincom_time_decay_3_1_10,lincom_sub_coef_3_10_1,lincom_time_decay_3_10_1,lincom_sub_coef_10_1_3,lincom_time_decay_10_1_3,lincom_sub_coef_10_3_1,lincom_time_decay_10_3_1,lincom_sub_coef_0.5_10_0.5,lincom_time_decay_0.5_10_0.5,lincom_sub_coef_10_0.5_0.5,lincom_time_decay_10_0.5_0.5,lincom_sub_coef_0.5_0.5_10,lincom_time_decay_0.5_0.5_10,agg_num_cosub_sum,agg_num_cosub_mean,agg_num_cosub_max,agg_num_cosub_min,agg_coclick_sub_coef_sum,agg_coclick_sub_coef_mean,agg_coclick_sub_coef_max,agg_coclick_sub_coef_min,agg_cocart_sub_coef_sum,agg_cocart_sub_coef_mean,agg_cocart_sub_coef_max,agg_cocart_sub_coef_min,agg_coorder_sub_coef_sum,agg_coorder_sub_coef_mean,agg_coorder_sub_coef_max,agg_coorder_sub_coef_min,agg_coclick_time_decay_sum,agg_coclick_time_decay_mean,agg_coclick_time_decay_max,agg_coclick_time_decay_min,agg_cocart_time_decay_sum,agg_cocart_time_decay_mean,agg_cocart_time_decay_max,agg_cocart_time_decay_min,agg_coorder_time_decay_sum,agg_coorder_time_decay_mean,agg_coorder_time_decay_max,agg_coorder_time_decay_min,agg_num_appearance_sum,agg_num_appearance_mean,agg_num_appearance_max,agg_num_appearance_min,agg_num_in_k_most_recent_items_sum,agg_num_in_k_most_recent_items_mean,agg_num_in_k_most_recent_items_max,agg_num_in_k_most_recent_items_min,agg_num_happend_later_sum,agg_num_happend_later_mean,agg_num_happend_later_max,agg_num_happend_later_min,agg_num_happend_before_sum,agg_num_happend_before_mean,agg_num_happend_before_max,agg_num_happend_before_min,agg_happend_later_ratio_sum,agg_happend_later_ratio_mean,agg_happend_later_ratio_max,agg_happend_later_ratio_min,agg_last_interact_sum,agg_last_interact_mean,agg_last_interact_max,agg_last_interact_min,agg_cointeractions_sub_coef_sum_sum,agg_cointeractions_sub_coef_sum_mean,agg_cointeractions_sub_coef_sum_max,agg_cointeractions_sub_coef_sum_min,agg_cointeractions_time_decay_sum_sum,agg_cointeractions_time_decay_sum_mean,agg_cointeractions_time_decay_sum_max,agg_cointeractions_time_decay_sum_min,agg_lincom_sub_coef_1_3_10_sum,agg_lincom_sub_coef_1_3_10_mean,agg_lincom_sub_coef_1_3_10_max,agg_lincom_sub_coef_1_3_10_min,agg_lincom_time_decay_1_3_10_sum,agg_lincom_time_decay_1_3_10_mean,agg_lincom_time_decay_1_3_10_max,agg_lincom_time_decay_1_3_10_min,agg_lincom_sub_coef_1_10_3_sum,agg_lincom_sub_coef_1_10_3_mean,agg_lincom_sub_coef_1_10_3_max,agg_lincom_sub_coef_1_10_3_min,agg_lincom_time_decay_1_10_3_sum,agg_lincom_time_decay_1_10_3_mean,agg_lincom_time_decay_1_10_3_max,agg_lincom_time_decay_1_10_3_min,agg_lincom_sub_coef_3_1_10_sum,agg_lincom_sub_coef_3_1_10_mean,agg_lincom_sub_coef_3_1_10_max,agg_lincom_sub_coef_3_1_10_min,agg_lincom_time_decay_3_1_10_sum,agg_lincom_time_decay_3_1_10_mean,agg_lincom_time_decay_3_1_10_max,agg_lincom_time_decay_3_1_10_min,agg_lincom_sub_coef_3_10_1_sum,agg_lincom_sub_coef_3_10_1_mean,agg_lincom_sub_coef_3_10_1_max,agg_lincom_sub_coef_3_10_

# Train params

In [8]:
from meow_utils import *
from catboost.utils import eval_metric

PARAMS = {
    'learning_rate': 0.5,
    'iterations': 1000,
    'early_stopping_rounds' : 20,
    'max_bin': 256,
    'depth': 7,
    'eval_metric': 'RecallAt:top=20;border=0.5',
    'use_best_model': True,
    'thread_count': -1,
}
N_SPLITS = 5
NUM_SAMPLES = 2
MODEL_PATH = '/home/anhphantq/meowmeow'
VERSION = '12'

# Train for each type

In [9]:
from meow_utils import get_len_group

In [10]:
import pandas as pd 
import numpy as np
tar = pd.read_parquet('/home/anhphantq/otto/splitted_data/test_labels.parquet')
tar = tar.loc[(tar['session'] <= candidates['user'].max()) & (tar['session'] >= candidates['user'].min()) ]
aids = tar.ground_truth.explode().astype('int32').rename('item')
tar = tar[['session', 'type']].rename({'session':'user'},axis=1)
tar = tar.merge(aids, left_index=True, right_index=True, how='left')
tar['type'] = tar['type'].apply(lambda x: 0 if x == 'clicks' else 1 if x == 'carts' else 2)
tar['label'] = 1
tar['item'] = tar['item'] + max_session
tar['user_type'] = tar['user'] * 10 + tar['type']

In [11]:
labels = []
users = candidates.user.values
items = candidates.item.values
from tqdm.notebook import tqdm
for i in tqdm(range(3)):
  cnt = 0
  tar_small = tar[tar['type'] == i].sort_values(['user', 'item']).set_index(['user', 'item'])
  
  tar_small_index = set(tar_small.index.tolist())
  label = []

  for i in tqdm(range(candidates.shape[0]), leave = False):
    if (users[i], items[i]) in tar_small_index:
      label.append(1)
      cnt += 1
    else: 
      label.append(0)
  labels.append(label)
  print(f'Found {cnt} positives', end = ' ')
del tar, tar_small_index
  



  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/55994691 [00:00<?, ?it/s]

Found 429115 positives 

  0%|          | 0/55994691 [00:00<?, ?it/s]

Found 106426 positives 

  0%|          | 0/55994691 [00:00<?, ?it/s]

Found 82634 positives 

In [12]:
labels = np.array(labels)

In [13]:
from sklearn.model_selection import GroupKFold
import os
import matplotlib.pyplot as plt
import shutil
from meow_utils import plot_importance
from catboost import CatBoostRanker, Pool
import seaborn as sns
sns.set()
%matplotlib inline
def train(t, id_type, draw_importance = False):
  global candidates, labels
  name = f'/{t}_only_'
  try:
    shutil.rmtree(MODEL_PATH + name + VERSION)
  except:
    pass
  else:
    print('remove old version')
  fig, axs = plt.subplots(1, N_SPLITS, figsize = (300, 30))
  skf = GroupKFold(n_splits=N_SPLITS)
  
  
  X_all = candidates.iloc[:, 2 :]
  y_all = labels[id_type]
  print('Building pool...')
  data_all = Pool(X_all, y_all, thread_count = os.cpu_count())
  print('Pool builed!!!!!!')
  for fold,(train_idx, valid_idx) in enumerate(skf.split(candidates, labels[id_type], groups=candidates['user'] )):
      neg_index = np.ravel(np.argwhere(labels[id_type][train_idx] == 0))
      pos_index = np.ravel(np.argwhere(labels[id_type][train_idx] == 1))
      #downsample many times
      for sample in range(NUM_SAMPLES):
        print(f'FOLD: {fold} Sample: {sample}')
        n_index = np.random.permutation(neg_index)[:len(pos_index) * 20]
        index = np.union1d(n_index, pos_index)
        
        data_train = data_all.slice(train_idx).slice(index)
        data_train.set_group_id(candidates['user'].values.astype(int)[train_idx][index])
        data_valid = data_all.slice(valid_idx)
        data_valid.set_group_id(candidates['user'].values.astype(int)[valid_idx])

        params = {'thread_count' : os.cpu_count(),
        **PARAMS}
        model = CatBoostRanker(**params)
        
        model.fit(data_train, eval_set = data_valid)
        
        try:
          os.mkdir(MODEL_PATH + name + VERSION)
        except:
          pass
        if fold < 5 and sample == 0 and draw_importance:
          plot_importance(X_all.columns.tolist(), model.get_feature_importance(data = data_valid, type = 'LossFunctionChange'), ax = axs[fold])
          axs[fold].bar_label(axs[fold].containers[0])
        model.save_model(MODEL_PATH + name + VERSION + f'/meowmewo_fold{fold}_sample{sample}')
  


In [14]:
for id_type, t in enumerate(['clicks', 'carts', 'orders']):
  train(t, id_type, draw_importance = False)

Building pool...
Pool builed!!!!!!
FOLD: 0 Sample: 0
0:	learn: 0.9996265	test: 0.8427769	best: 0.8427769 (0)	total: 1.97s	remaining: 32m 49s
1:	learn: 0.9996682	test: 0.8689877	best: 0.8689877 (1)	total: 3.53s	remaining: 29m 19s
2:	learn: 0.9997064	test: 0.8838731	best: 0.8838731 (2)	total: 5.02s	remaining: 27m 47s
3:	learn: 0.9999288	test: 0.8993414	best: 0.8993414 (3)	total: 6.56s	remaining: 27m 14s
4:	learn: 0.9999792	test: 0.9083074	best: 0.9083074 (4)	total: 8.03s	remaining: 26m 38s
5:	learn: 0.9999757	test: 0.9113816	best: 0.9113816 (5)	total: 9.83s	remaining: 27m 8s
6:	learn: 0.9999931	test: 0.9178493	best: 0.9178493 (6)	total: 11.6s	remaining: 27m 30s
7:	learn: 0.9999948	test: 0.9206390	best: 0.9206390 (7)	total: 13.1s	remaining: 27m 8s
8:	learn: 0.9999931	test: 0.9214509	best: 0.9214509 (8)	total: 14.8s	remaining: 27m 7s
9:	learn: 0.9999931	test: 0.9227070	best: 0.9227070 (9)	total: 16.2s	remaining: 26m 44s
10:	learn: 0.9999913	test: 0.9243864	best: 0.9243864 (10)	total: 17.7s

Training has stopped (degenerate solution on iteration 38, probably too small l2-regularization, try to increase it)



bestTest = 0.9893442255
bestIteration = 33

Shrink model to first 34 iterations.
FOLD: 2 Sample: 1
0:	learn: 0.9999985	test: 0.9737862	best: 0.9737862 (0)	total: 585ms	remaining: 9m 44s
1:	learn: 0.9999986	test: 0.9821660	best: 0.9821660 (1)	total: 1.15s	remaining: 9m 32s
2:	learn: 0.9999986	test: 0.9849560	best: 0.9849560 (2)	total: 1.73s	remaining: 9m 35s
3:	learn: 0.9999986	test: 0.9865112	best: 0.9865112 (3)	total: 2.32s	remaining: 9m 38s
4:	learn: 0.9999986	test: 0.9872485	best: 0.9872485 (4)	total: 2.9s	remaining: 9m 37s
5:	learn: 0.9999986	test: 0.9877963	best: 0.9877963 (5)	total: 3.52s	remaining: 9m 42s
6:	learn: 0.9999986	test: 0.9883265	best: 0.9883265 (6)	total: 4.13s	remaining: 9m 45s
7:	learn: 0.9999987	test: 0.9886130	best: 0.9886130 (7)	total: 4.79s	remaining: 9m 53s
8:	learn: 0.9999987	test: 0.9886880	best: 0.9886880 (8)	total: 5.43s	remaining: 9m 58s
9:	learn: 0.9999987	test: 0.9889345	best: 0.9889345 (9)	total: 6.05s	remaining: 9m 58s
10:	learn: 0.9999987	test: 0.98

Training has stopped (degenerate solution on iteration 30, probably too small l2-regularization, try to increase it)



bestTest = 0.9972653872
bestIteration = 20

Shrink model to first 21 iterations.
FOLD: 0 Sample: 1
0:	learn: 0.9999982	test: 0.9913844	best: 0.9913844 (0)	total: 521ms	remaining: 8m 40s
1:	learn: 0.9999983	test: 0.9942585	best: 0.9942585 (1)	total: 1.02s	remaining: 8m 27s
2:	learn: 0.9999983	test: 0.9955044	best: 0.9955044 (2)	total: 1.54s	remaining: 8m 31s
3:	learn: 0.9999984	test: 0.9961884	best: 0.9961884 (3)	total: 2.09s	remaining: 8m 39s
4:	learn: 0.9999984	test: 0.9964021	best: 0.9964021 (4)	total: 2.61s	remaining: 8m 39s
5:	learn: 0.9999985	test: 0.9966340	best: 0.9966340 (5)	total: 3.15s	remaining: 8m 42s
6:	learn: 0.9999984	test: 0.9968106	best: 0.9968106 (6)	total: 3.71s	remaining: 8m 46s
7:	learn: 0.9999985	test: 0.9968629	best: 0.9968629 (7)	total: 4.24s	remaining: 8m 45s
8:	learn: 0.9999985	test: 0.9970193	best: 0.9970193 (8)	total: 4.83s	remaining: 8m 52s
9:	learn: 0.9999985	test: 0.9970174	best: 0.9970193 (8)	total: 5.42s	remaining: 8m 56s
10:	learn: 0.9999985	test: 0.9

Training has stopped (degenerate solution on iteration 45, probably too small l2-regularization, try to increase it)



bestTest = 0.9973426456
bestIteration = 34

Shrink model to first 35 iterations.
FOLD: 4 Sample: 0
0:	learn: 0.9999986	test: 0.9915803	best: 0.9915803 (0)	total: 516ms	remaining: 8m 35s
1:	learn: 0.9999988	test: 0.9949513	best: 0.9949513 (1)	total: 1.02s	remaining: 8m 27s
2:	learn: 0.9999988	test: 0.9958663	best: 0.9958663 (2)	total: 1.68s	remaining: 9m 19s
3:	learn: 0.9999989	test: 0.9961104	best: 0.9961104 (3)	total: 2.33s	remaining: 9m 40s
4:	learn: 0.9999989	test: 0.9965882	best: 0.9965882 (4)	total: 2.99s	remaining: 9m 55s
5:	learn: 0.9999989	test: 0.9967632	best: 0.9967632 (5)	total: 3.62s	remaining: 9m 59s
6:	learn: 0.9999989	test: 0.9968185	best: 0.9968185 (6)	total: 4.31s	remaining: 10m 11s
7:	learn: 0.9999989	test: 0.9968250	best: 0.9968250 (7)	total: 5.03s	remaining: 10m 23s
8:	learn: 0.9999989	test: 0.9968764	best: 0.9968764 (8)	total: 5.77s	remaining: 10m 34s
9:	learn: 0.9999990	test: 0.9969003	best: 0.9969003 (9)	total: 6.46s	remaining: 10m 39s
10:	learn: 0.9999990	test: